<a href="https://colab.research.google.com/github/girish445ai/Recurrent_Neural_networks/blob/main/Transliteration_without_attention_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries 

In [1]:
import io
import numpy as np
import tensorflow 
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, GRU, Dropout, SimpleRNN
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from math import log
from numpy import array
from numpy import argmax

import keras
from keras.models import load_model
from math import log1p 

In [2]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 49.5 MB/s 
     |████████████████████████████████| 181 kB 51.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


### Unzipping the dataset

Lexicons for Latin-Telugu are taken from Google's Dakshina dataset.

In [3]:
# Downloading dakshina dataset
!yes | wget "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

--2022-05-05 09:03:21--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.159.128, 74.125.126.128, 74.125.70.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.159.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   131MB/s    in 11s     

2022-05-05 09:03:32 (171 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [4]:
# Unzipping dataset
!yes | tar xopf dakshina_dataset_v1.0.tar

In [5]:
# The folder containing the datasets to be used in this program
!ls dakshina_dataset_v1.0/te/lexicons

te.translit.sampled.dev.tsv   te.translit.sampled.train.tsv
te.translit.sampled.test.tsv


In [6]:
print_data = True

## Reading the dataset


In [7]:
train_path = "./dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.train.tsv"
dev_path = "./dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.dev.tsv"
test_path = "./dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.dev.tsv"

def reading_data(corpus_file):
  # function reads the raw text of words and returns native versions of words
  telugu_words = []
  latin_words = []
  with io.open(corpus_file, encoding ='utf-8') as f:
    for line in f:
      if '\t' not in line:
        continue
      tokens = line.rstrip().split("\t")
      latin_words.append(tokens[1])
      telugu_words.append(tokens[0])
  return latin_words, telugu_words

train_source, train_target = reading_data(train_path)
val_source, val_target = reading_data(dev_path)
test_source, test_target = reading_data(test_path)

print("Number of training samples: ", len(train_source))
print("Number of validation samples: ", len(val_source))
print("Number of testing samples: ", len(test_source))

Number of training samples:  58550
Number of validation samples:  5683
Number of testing samples:  5683


In [8]:
arr = np.arange(len(train_source))
np.random.shuffle(arr)
arr1 = np.arange(len(val_source))
np.random.shuffle(arr1)

input_chars = set()
target_chars = set()
input_texts_ns = []
target_texts_ns = []
val_input_texts_ns = []
val_target_texts_ns = []

for (input_text, target_text) in zip(train_source, train_target):
    # "tab" is the "start sequence" characte ,"\n" is "end sequence" character.
    target_text = "B" + target_text + "E"
    input_texts_ns.append(input_text)
    target_texts_ns.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

for (input_text, target_text) in zip(val_source, val_target):
    # "tab" is the "start sequence" characte ,"\n" is "end sequence" character.
    target_text = "B" + target_text + "E"
    val_input_texts_ns.append(input_text)
    val_target_texts_ns.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_texts = []
target_texts = []

for i in range(len(train_source)):
    input_texts.append(input_texts_ns[arr[i]])
    target_texts.append(target_texts_ns[arr[i]])

val_input_texts = []
val_target_texts = []

for i in range(len(val_source)):
    val_input_texts.append(val_input_texts_ns[arr1[i]])
    val_target_texts.append(val_target_texts_ns[arr1[i]])

input_chars.add(" ")
target_chars.add(" ")

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))


no_enc_tokens = len(input_chars)
no_dec_tokens = len(target_chars)
enc_seq_length = max([len(txt) for txt in input_texts])
dec_seq_length = max([len(txt) for txt in target_texts])
val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])



print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", no_enc_tokens)
print("Number of unique output tokens:", no_dec_tokens)
print("Max sequence length for inputs:", enc_seq_length)
print("Max sequence length for outputs:", dec_seq_length)
print("Max sequence length for val inputs:", val_max_encoder_seq_length)
print("Max sequence length for val outputs:", val_max_decoder_seq_length)

print(input_chars)
print(target_chars)

Number of samples: 58550
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 25
Max sequence length for outputs: 22
Max sequence length for val inputs: 21
Max sequence length for val outputs: 21
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[' ', 'B', 'E', 'ం', 'ః', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఎ', 'ఏ', 'ఐ', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'వ', 'శ', 'ష', 'స', 'హ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ె', 'ే', 'ై', 'ొ', 'ో', 'ౌ', '్', '\u200c']


In [9]:
print(input_texts[123:130])
print(target_texts[123:130])

['chesukoka', 'gurthincha', 'haindava', 'adugupettina', 'massachusetts', 'untundane', 'paalaace']
['BచేసుకోకE', 'Bగుర్తించE', 'BహైందవE', 'Bఅడుగుపెట్టినE', 'Bమసాచుసెట్స్E', 'BఉంటుందనేE', 'Bప్యాలెస్E']


**Training** :

In [10]:
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])
print(input_token_index)
print(target_token_index)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{' ': 0, 'B': 1, 'E': 2, 'ం': 3, 'ః': 4, 'అ': 5, 'ఆ': 6, 'ఇ': 7, 'ఈ': 8, 'ఉ': 9, 'ఊ': 10, 'ఋ': 11, 'ఎ': 12, 'ఏ': 13, 'ఐ': 14, 'ఒ': 15, 'ఓ': 16, 'ఔ': 17, 'క': 18, 'ఖ': 19, 'గ': 20, 'ఘ': 21, 'చ': 22, 'ఛ': 23, 'జ': 24, 'ఝ': 25, 'ఞ': 26, 'ట': 27, 'ఠ': 28, 'డ': 29, 'ఢ': 30, 'ణ': 31, 'త': 32, 'థ': 33, 'ద': 34, 'ధ': 35, 'న': 36, 'ప': 37, 'ఫ': 38, 'బ': 39, 'భ': 40, 'మ': 41, 'య': 42, 'ర': 43, 'ఱ': 44, 'ల': 45, 'ళ': 46, 'వ': 47, 'శ': 48, 'ష': 49, 'స': 50, 'హ': 51, 'ా': 52, 'ి': 53, 'ీ': 54, 'ు': 55, 'ూ': 56, 'ృ': 57, 'ె': 58, 'ే': 59, 'ై': 60, 'ొ': 61, 'ో': 62, 'ౌ': 63, '్': 64, '\u200c': 65}


In [11]:
# Encoder Input Sequences are padded to a maximum length of MAX encoder SeqLen characters. 
enc_input_data = np.zeros(
    (len(input_texts), enc_seq_length), dtype="float32"
)
dec_input_data = np.zeros(
    (len(input_texts), dec_seq_length), dtype="float32"
)
dec_target_data = np.zeros(
    (len(input_texts), dec_seq_length, no_dec_tokens), dtype="float32"
)
#Decoder Target Sequences are Padded to a maximum length of max_decoder SeqLen characters with a vocabulary of sizeofTeluguVocab different characters. 
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        enc_input_data[i, t] = input_token_index[char]
    enc_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        # dec_target_data is ahead of dec_input_data by one timestep
        dec_input_data[i, t] = target_token_index[char]
        if t > 0:
            # dec_target_data will not include the start character.
            dec_target_data[i, t - 1, target_token_index[char]] = 1.0
    dec_input_data[i, t + 1: ] = target_token_index[" "]
    dec_target_data[i, t:, target_token_index[" "]] = 1.0


val_enc_input_data = np.zeros(
    (len(input_texts), val_max_encoder_seq_length), dtype="float32"
)
val_dec_input_data = np.zeros(
    (len(input_texts), val_max_decoder_seq_length), dtype="float32"
)
val_dec_target_data = np.zeros(
    (len(input_texts), val_max_decoder_seq_length, no_dec_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(input_text):
        val_enc_input_data[i, t] = input_token_index[char]
    val_enc_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        # dec_target_data is ahead of decoder_input_data by one timestep
        val_dec_input_data[i, t] = target_token_index[char]
        if t > 0:
            # dec_target_data will be ahead by one timestep
            # and will not include the start character.
            val_dec_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_dec_input_data[i, t + 1: ] = target_token_index[" "]
    val_dec_target_data[i, t:, target_token_index[" "]] = 1.0




In [12]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
print(reverse_target_char_index)

{0: ' ', 1: 'B', 2: 'E', 3: 'ం', 4: 'ః', 5: 'అ', 6: 'ఆ', 7: 'ఇ', 8: 'ఈ', 9: 'ఉ', 10: 'ఊ', 11: 'ఋ', 12: 'ఎ', 13: 'ఏ', 14: 'ఐ', 15: 'ఒ', 16: 'ఓ', 17: 'ఔ', 18: 'క', 19: 'ఖ', 20: 'గ', 21: 'ఘ', 22: 'చ', 23: 'ఛ', 24: 'జ', 25: 'ఝ', 26: 'ఞ', 27: 'ట', 28: 'ఠ', 29: 'డ', 30: 'ఢ', 31: 'ణ', 32: 'త', 33: 'థ', 34: 'ద', 35: 'ధ', 36: 'న', 37: 'ప', 38: 'ఫ', 39: 'బ', 40: 'భ', 41: 'మ', 42: 'య', 43: 'ర', 44: 'ఱ', 45: 'ల', 46: 'ళ', 47: 'వ', 48: 'శ', 49: 'ష', 50: 'స', 51: 'హ', 52: 'ా', 53: 'ి', 54: 'ీ', 55: 'ు', 56: 'ూ', 57: 'ృ', 58: 'ె', 59: 'ే', 60: 'ై', 61: 'ొ', 62: 'ో', 63: 'ౌ', 64: '్', 65: '\u200c'}


In [13]:
print(enc_input_data[1])
print(dec_input_data[1])
print(dec_target_data[1])

[18. 15.  7.  1. 13. 21. 12.  1. 14. 21.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
[ 1. 43. 62. 20. 41. 55. 45. 36. 55.  2.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


For Validation and testing:

In [14]:
print(val_dec_input_data[26])
print(val_dec_target_data[26])

[ 1.  5.  3. 34. 55. 36. 52.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [15]:
x_test = val_enc_input_data
y_test = val_target_texts

## MODEL TRAINING 

In [16]:
class MyRNN(object):
  def __init__(self,cell_type = 'RNN',in_emb = 32, hidden_size=32, learning_rate= 1e-3, 
               dropout=0.4,pred_type ='greedy',epochs = 10, batch_size = 32,beam_width = 5,
               num_enc = 1,num_dec = 1):
    
    self.cell_type = cell_type
    self.in_emb = in_emb
    self.hidden_size = hidden_size
    self.learning_rate = learning_rate
    self.dropout = dropout
    self.pred_type = pred_type
    self.epochs = epochs
    self.batch_size = batch_size
    self.beam_width = beam_width
    self.num_enc = num_enc
    self.num_dec = num_dec

  def build_fit(self,enc_input_data,dec_input_data,dec_target_data,x_test, y_test):
    enc_inputs = Input(shape=(None, ),name = 'Enc_inputs')

    # Add an Embedding layer expecting input vocab of size no_enc_tokens, and
    # output embedding dimension of size in_enc.
    enc_emb =  Embedding(no_enc_tokens, self.in_emb , mask_zero = True,name = 'Enc_emb')(enc_inputs)

    enc_outputs = enc_emb
    if self.cell_type == 'LSTM':
      encoder_lstm = LSTM(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name="Enc_hidden_1")
      enc_outputs, state_h, state_c = encoder_lstm(enc_outputs)
      encoder_states = [state_h, state_c]

      # Add a LSTM layer with hidden_size internal units.
      for i in range( 2, self.num_enc +1):
        layer_name = ('Enc_hidden_%d') %i

        encoder_lstm = LSTM(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name=layer_name)
        enc_outputs, state_h, state_c = encoder_lstm(enc_outputs,initial_state = encoder_states)
        encoder_states = [state_h, state_c]

    elif self.cell_type == 'GRU':
      encoder_gru = GRU(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name="Enc_hidden_1")
      enc_outputs, state_h = encoder_gru(enc_outputs)
      encoder_states = [state_h]

      for i in range(2, self.num_enc +1):
        layer_name = ('Enc_hidden_%d') %i
        encoder_gru = GRU(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name=layer_name)
        enc_outputs, state_h = encoder_gru(enc_outputs, initial_state = encoder_states)
        encoder_states = [state_h]  

    elif self.cell_type == 'RNN':
      encoder_rnn = SimpleRNN(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name="Enc_hidden_1")
      enc_outputs, state_h = encoder_rnn(enc_outputs)
      encoder_states = [state_h]

      for i in range(2, self.num_enc +1):
        layer_name = ('Enc_hidden_%d') %i
        encoder_rnn = SimpleRNN(self.hidden_size, return_state=True,dropout = self.dropout, return_sequences=True, name=layer_name)
        enc_outputs, state_h = encoder_rnn(enc_outputs, initial_state = encoder_states)
        encoder_states = [state_h]  

    # Set up the decoder, using `encoder_states` as initial state.
    dec_inputs = Input(shape=(None,), name = 'Dec_inputs')
    dec_emb_layer = Embedding(no_dec_tokens, self.hidden_size, mask_zero = True, name = 'Dec_emb')
    dec_emb = dec_emb_layer(dec_inputs)
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    dec_outputs = dec_emb
    if self.cell_type == 'LSTM':
      decoder_lstm = LSTM(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name="Dec_hidden_1")
      dec_outputs, _, _ = decoder_lstm(dec_outputs, initial_state = encoder_states)
      
      for i in range(2, self.num_dec +1):
        layer_name = ('Dec_hidden_%d') %i

        decoder_lstm = LSTM(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name=layer_name)
        dec_outputs, _, _ = decoder_lstm(dec_outputs, initial_state = encoder_states)

    elif self.cell_type == 'GRU':
      decoder_gru = GRU(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name="Dec_hidden_1")
      dec_outputs, _ = decoder_gru(dec_outputs, initial_state = encoder_states)

      for i in range(2, self.num_dec+1):
        layer_name = ('Dec_hidden_%d') %i
        decoder_gru = GRU(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name=layer_name)
        dec_outputs, _ = decoder_gru(dec_outputs, initial_state = encoder_states)

    elif self.cell_type == 'RNN':
      decoder_rnn = SimpleRNN(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name="Dec_hidden_1")
      dec_outputs, _ = decoder_rnn(dec_outputs, initial_state = encoder_states)

      for i in range(2, self.num_dec+1):
        layer_name = ('Dec_hidden_%d') %i
        decoder_rnn = SimpleRNN(self.hidden_size, return_sequences=True, return_state=True,dropout = self.dropout, name=layer_name)
        dec_outputs, _ = decoder_rnn(dec_outputs, initial_state = encoder_states)

    decoder_dense = Dense(no_dec_tokens, activation='softmax', name = 'dense')
    dec_outputs = decoder_dense(dec_outputs)

    # Define the model that takes encoder and decoder input 
    # to output dec_outputs
    model = Model([enc_inputs, dec_inputs], dec_outputs)

    model.summary()

    # Define the optimizer
    optimizer = Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999)
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])
  
    model.fit(
        [enc_input_data, dec_input_data],
        dec_target_data,
        batch_size=self.batch_size,
        epochs=self.epochs,
        callbacks = [WandbCallback()]
        )
    
    encoder_model,decoder_model = self.inference_model(model)
 
    global_total = 0
    global_correct = 0
    for i in range(len(val_source)):
      #input_seq = val_enc_input_data[i : i + 1]
      input_seq = x_test[i : i + 1]
      result = self.decode_sequence(encoder_model,decoder_model,input_seq)
      #target = val_target_texts[i]
      target = y_test[i]
      target = target[1:len(target)-1]
      result = result[0:len(result)-1]
      #print("Target: %s \n Result: %s" % (target, result))

      if result.strip() == target.strip():
        global_correct = global_correct + 1
      
      global_total = global_total + 1
      accuracy_epoch = global_correct/global_total
      if global_total % 50 == 0:
        wandb.log({'epoch_accuracy' : accuracy_epoch})
      #print("Accuracy: %s" % (accuracy_epoch))
    
    val_accuracy = global_correct/global_total
    #print(val_accuracy)


    wandb.log({'val_accuracy' : val_accuracy})
    
  def inference_model(self,model):
    enc_inputs = model.input[0]  # input_1
    if self.cell_type == 'RNN' or self.cell_type == 'GRU':
      enc_outputs, state_h_enc = model.get_layer('Enc_hidden_'+ str(self.num_enc)).output
      encoder_states = [state_h_enc]
      encoder_model = Model(enc_inputs, encoder_states)

      dec_inputs = model.input[1]  # input_1
      dec_outputs = model.get_layer('Dec_emb')(dec_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for i in range(1,self.num_dec +1):
        decoder_state_input_h = keras.Input(shape=(self.hidden_size,))
        curr_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('Dec_hidden_'+ str(i))
        dec_outputs, state_h_dec = decoder(dec_outputs, initial_state=curr_states_inputs)

        decoder_states += [state_h_dec]
        decoder_states_inputs += curr_states_inputs

    elif self.cell_type == 'LSTM':
      enc_outputs, state_h_enc, state_c_enc = model.get_layer('Enc_hidden_'+ str(self.num_enc)).output  # lstm_1
      encoder_states = [state_h_enc, state_c_enc]
      encoder_model = Model(enc_inputs, encoder_states)

      dec_inputs = model.input[1]  # input_1
      dec_outputs = model.get_layer('Dec_emb')(dec_inputs)
      decoder_states_inputs = []
      decoder_states = []

      for i in range(1,self.num_dec +1):
        decoder_state_input_h = keras.Input(shape=(self.hidden_size,))
        decoder_state_input_c = keras.Input(shape=(self.hidden_size,))
        curr_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder = model.get_layer('Dec_hidden_'+ str(i))
        dec_outputs, state_h_dec, state_c_dec = decoder(dec_outputs, initial_state=curr_states_inputs)

        decoder_states += [state_h_dec, state_c_dec]
        decoder_states_inputs += curr_states_inputs


    decoder_dense = model.get_layer('dense')
    dec_outputs = decoder_dense(dec_outputs)
    decoder_model = Model([dec_inputs] + decoder_states_inputs, [dec_outputs] + decoder_states)

    return encoder_model,decoder_model

  def decode_sequence(self,encoder_model,decoder_model,input_seq):
    # Encode the input as state vectors.
    states_value = [encoder_model.predict(input_seq)] * self.num_dec
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['B']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        if self.cell_type == 'RNN' or self.cell_type == 'GRU':
          dummy = decoder_model.predict([target_seq] + [states_value])
          output_tokens, states_value = dummy[0],dummy[1:]
          
        elif self.cell_type == 'LSTM':  
          dummy = decoder_model.predict([target_seq] + states_value)
          output_tokens, states_value = dummy[0],dummy[1:]
        
        #print(output_tokens[0,:,:])
        if self.pred_type == 'greedy':
          beam_w = 1
        elif self.pred_type == 'beam_search':
          beam_w = self.beam_width
        sampled_token_index = self.beam_search_decoder(output_tokens[0,:,:], beam_w)
        sampled_token_index = sampled_token_index[beam_w-1][0]

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit when reaches max length or stop character is encountered.
        if sampled_char == 'E' or len(decoded_sentence) > dec_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update state

    return decoded_sentence
  
  def beam_search_decoder(self,data, k):
    sequences = [[list(), 0.0]]
    # read each step in sequence
    for row in data:
      all_candidates = list()
      # expand each current candidate
      for i in range(len(sequences)):
        seq, score = sequences[i]
        for j in range(len(row)):
          candidate = [seq + [j], score - log(row[j])]
          all_candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      # select k best
      sequences = ordered[:k]
    return sequences

## Sweep

In [17]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {

        'dropout': {
            'values': [0.0, 0.1, 0.2]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {
            'values': [64, 128]
        },
        'in_emb': {
            'values': [32, 64, 128]
        },
        'num_enc': {
            'values': [1, 2, 3]
        },
        'num_dec': {
            'values': [1, 2, 3]
        },
        'hidden_size':{
            'values': [32, 64, 128]
        },
        'cell_type': {
            'values': ['RNN', 'GRU', 'LSTM']
        },
        'dec_search': {
            'values': ['beam_search', 'greedy']
        },
        'beam_width':{
            'values': [3,5]
        }
    }
}

In [18]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="girishrongali", project="assignment3")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 5sdtovzt
Sweep URL: https://wandb.ai/girishrongali/assignment3/sweeps/5sdtovzt


In [19]:
def train_sweep():
  config_defaults = {
        'dropout': 0.4,
        'learning_rate': 1e-3,
        'batch_size': 32,
        'epochs' : 15,
        'in_emb': 32,
        'num_enc': 2,
        'num_dec': 2,
        'hidden_size': 32,
        'cell_type': 'RNN',
        'dec_search': 'beam_search',
        'beam_width': 5
        }

  # Initialize a new wandb run
  wandb.init(config = config_defaults)
  
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config

  wandb.run.name = str(config.cell_type)+ '_' + config.dec_search+'_bs_'+str(config.batch_size)
  
  model_rnn = MyRNN(cell_type = config.cell_type, in_emb = config.in_emb, hidden_size=config.hidden_size,
                learning_rate= config.learning_rate, dropout=config.dropout,pred_type = config.dec_search,epochs = config.epochs,
                batch_size = config.batch_size, beam_width = config.beam_width, num_enc = config.num_enc,num_dec = config.num_dec)
  
  model_rnn.build_fit(enc_input_data,dec_input_data,dec_target_data,x_test, y_test)

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="girishrongali", project="assignment3")
wandb.agent(sweep_id, lambda : train_sweep())

Create sweep with ID: hxi6n58c
Sweep URL: https://wandb.ai/girishrongali/assignment3/sweeps/hxi6n58c


wandb: Agent Starting Run: dkqwwby8 with config:
wandb: 	batch_size: 64
wandb: 	beam_width: 3
wandb: 	cell_type: GRU
wandb: 	dec_search: beam_search
wandb: 	dropout: 0
wandb: 	hidden_size: 128
wandb: 	in_emb: 128
wandb: 	learning_rate: 0.001
wandb: 	num_dec: 1
wandb: 	num_enc: 1
wandb: Currently logged in as: girishrongali. Use `wandb login --relogin` to force relogin


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enc_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Enc_emb (Embedding)            (None, None, 128)    3456        ['Enc_inputs[0][0]']             
                                                                                                  
 Dec_emb (Embedding)            (None, None, 128)    8448        ['Dec_inputs[0][0]']             
                                                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
915/915 [==============================] - 35s 25ms/step - loss: 0.9428 - accuracy: 0.4298 - _timestamp: 1651741727.0000 - _runtime: 46.0000
Epoch 2/15
915/915 [==============================] - 23s 25ms/step - loss: 0.5004 - accuracy: 0.6845 - _timestamp: 1651741750.0000 - _runtime: 69.0000
Epoch 3/15
915/915 [==============================] - 23s 25ms/step - loss: 0.2898 - accuracy: 0.8133 - _timestamp: 1651741773.0000 - _runtime: 92.0000
Epoch 4/15
915/915 [==============================] - 23s 25ms/step - loss: 0.2098 - accuracy: 0.8642 - _timestamp: 1651741796.0000 - _runtime: 115.0000
Epoch 5/15
915/915 [==============================] - 23s 25ms/step - loss: 0.1699 - accuracy: 0.8892 - _timestamp: 1651741818.0000 - _runtime: 137.0000
Epoch 6/15
915/915 [==============================] - 23s 25ms/step - loss: 0.1456 - accuracy: 0.9049 - _timestamp: 1651741841.0000 - _runtime: 160.0000
Epoch 7/15
915/915 [==============================] - 23s 25ms/step - loss: 0.1286 - 

...

wandb: Agent Starting Run: 089hp274 with config:
wandb: 	batch_size: 128
wandb: 	beam_width: 5
wandb: 	cell_type: LSTM
wandb: 	dec_search: beam_search
wandb: 	dropout: 0.1
wandb: 	hidden_size: 128
wandb: 	in_emb: 32
wandb: 	learning_rate: 0.0001
wandb: 	num_dec: 1
wandb: 	num_enc: 3


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enc_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Enc_emb (Embedding)            (None, None, 32)     864         ['Enc_inputs[0][0]']             
                                                                                                  
 Enc_hidden_1 (LSTM)            [(None, None, 128),  82432       ['Enc_emb[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇██
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch_accuracy,▁▁▁▁▄▃▃▄▄▄▇▇▇▆███▇▇▇▆▆▆▆▅▆▆▆▆▆▇▇▇▆▆▆▆▆▆▆
loss,█▅▄▄▃▃▃▃▂▂▂▂▁▁▁
val_accuracy,▁
accuracy,0.55459
epoch,14
epoch_accuracy,0.00283
loss,0.70599
val_accuracy,0.00282


wandb: Agent Starting Run: d6wf4wi8 with config:
wandb: 	batch_size: 64
wandb: 	beam_width: 5
wandb: 	cell_type: GRU
wandb: 	dec_search: beam_search
wandb: 	dropout: 0.1
wandb: 	hidden_size: 128
wandb: 	in_emb: 32
wandb: 	learning_rate: 0.0001
wandb: 	num_dec: 1
wandb: 	num_enc: 3


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enc_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Enc_emb (Embedding)            (None, None, 32)     864         ['Enc_inputs[0][0]']             
                                                                                                  
 Enc_hidden_1 (GRU)             [(None, None, 128),  62208       ['Enc_emb[0][0]']                
                                 (None, 128)]                                                     
                                                                                                  
 Dec_inputs (InputLayer)        [(None, None)]       0           []                           